# Etude des bandes de phonons accoustiques du cristal de $Sr_2NCl$

#### Raphaël Roekens 70732200

Ce notebook sert à visualiser la structure de bandes de phonons. Plus particulièrement, on s'intéressera aux bandes accoustiques.

## 1. Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mp_api.client import MPRester
from pymatgen.phonon.plotter import PhononBSPlotter
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.graph_objects import Scatter

In [124]:
mp_key ="iX81lxwugQ7QfGznstwLyMrndo3WUZbE"
mp_id = "mp-23033"

with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

## 2. Zone de brillouin

La zone de brillouin n'est pas lié au type de particule. Elle ne change donc pas pour des électrons ou des phonons

In [125]:
plot_brillouin_zone(ph_bs.structure)

c:\Users\roeke\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymatgen\symmetry\bandstructure.py:181: UserWarning:

The input structure does not match the expected standard primitive! The path may be incorrect. Use at your own risk.



## 3. Structure de bandes phoniques

Les branches acoustiques sont les branches de faibles fréquences. Plus précisémment, ce sont les branches qui tendent linéairement la fréquence nulle lorsque le nombre d'onde se rapporche de $\Gamma$.

In [ ]:
fig_ph_bs_and_dos = get_plot_bs_and_dos(ph_bs, ph_dos, plot_range=[0,10])
fig_ph_bs_and_dos.show()

## 4.  Vitesse du son

La vitesse du son est défini comme $$v_s = \frac{\partial \omega}{\partial q}$$ 

avec $\omega$ en $[\frac{rad}{s}]$, la pulsation de l'onde et $q $ $[\frac{rad}{m}]$, le vecteur d'onde. 

Cette formule n'est valable que pour les branches accoustiques et pour de petites valeurs de $q$, lorsque $\omega(q)$ est une relation linéaire. 

### 4.1 calcul de la vitesse

In [127]:
data = PhononBSPlotter(ph_bs).bs_plot_data()
f = data.get('frequency')
q = data.get('distances')

f_list = [[f[0][0][0], f[0][0][1]], [f[3][1][-2], f[3][1][-1]], [f[4][2][0], f[4][2][1]]]
q_list = [[q[0][0], q[0][1]],       [q[3][-2], q[3][-1]],       [q[4][0], q[4][1]]]
v_list = np.zeros(3)

for i in range(3):
    df = f_list[i][1] - f_list[i][0]
    dq = q_list[i][1] - q_list[i][0]
    v_list[i] = np.abs(2*np.pi*df/dq)*100





### 4.2 Visualisation

In [128]:
def get_extended_line(x1, y1, x2, y2, extension=2):
    m = (y2 - y1) / (x2 - x1)  # Slope
    b = y1 - m * x1  # Intercept
    x_values = np.linspace(x1 - extension, x2 + extension, 100)  # Extend line
    y_values = m * x_values + b
    return x_values, y_values

def add_scattered_points(fig, q_list, f_list, color):
    """
    Adds scattered points to the figure.
    """
    fig.add_trace(
        Scatter(
            x=[q[0] for q in q_list] + [q[1] for q in q_list],  # All x-coordinates
            y=[f[0] for f in f_list] + [f[1] for f in f_list],  # All y-coordinates
            mode='markers',
            marker=dict(color=['blue', 'red',  'green','blue','red', 'green'], size=10),  # Customize marker color and size
            name='Scattered Points'
        )
    )

def add_dashed_lines(fig, q_list, f_list, color,labels):
    """
    Adds dashed lines connecting the points and extending them.
    """
    for i in range(len(q_list)):
        x1, y1 = q_list[i][0], f_list[i][0]
        x2, y2 = q_list[i][1], f_list[i][1]
        x_values, y_values = get_extended_line(x1, y1, x2, y2)  # Get extended line coordinates
        
        fig.add_trace(
            Scatter(
                x=x_values,  # Extended x-coordinates
                y=y_values,  # Extended y-coordinates
                mode='lines',
                line=dict(color=color[2 * i], dash='dash'),  # Dashed line with corresponding color
                name=labels[i]
            )
        )

def zoom_into_region(fig, q, f, xleft=2, xright =2, y_padding=3):
    """
    Updates the figure to zoom into a specific region around the points.
    """
    x_min = min(q) - xleft
    x_max = max(q) + xright
    y_min = min(f)
    y_max = max(f) + y_padding
    
    fig.update_xaxes(range=[x_min, x_max])
    fig.update_yaxes(range=[y_min, y_max])



In [131]:
# Main Code
f_list = [[f[0][0][0], f[0][0][1]], [f[3][1][-2], f[3][1][-1]], [f[4][2][0], f[4][2][1]]]
q_list = [[q[0][0], q[0][1]],       [q[3][-2], q[3][-1]],       [q[4][0], q[4][1]]]

ph_bs.bands = ph_bs.bands[0:3]

fig_ph_bs = get_plot_bs(ph_bs, plot_range=[0, 5])
color = ['blue', 'blue', 'red', 'red', 'green', 'green']

labels = [
    f'Band 1 : vs = {v_list[0]:.3f} m/s',
    f'Band 2 : vs = {v_list[1]:.3f} m/s',
    f'Band 3 : vs = {v_list[2]:.3f} m/s'
]
# Add points and dashed lines
add_scattered_points(fig_ph_bs, q_list, f_list, color)
add_dashed_lines(fig_ph_bs, q_list, f_list, color,labels)

zoom_into_region(fig_ph_bs, q_list[0], f_list[0],xleft=0)
fig_ph_bs.show()

zoom_into_region(fig_ph_bs, q_list[1], f_list[1])
fig_ph_bs.show()


#### Commentaire :

Il y a trois courbes de dispersions différentes. La fréquence ne tend vers 0 que pour 3 directions différentes : $\Gamma-L$,  $Z-\Gamma$.   $\Gamma-X|Q$. Comme prévu, la vitesse est plus grande pour des droite plus raides.

## Source

Copylot a été utilisé pour générer les fonctions du point "4.2 Visualisation".